## AoC 2023 - Day xx

### Setup imports and get raw data

In [2]:
from collections import deque
#from icecream import ic
import pprint

#SEARCH_MIN = 7
#SEARCH_MAX = 27
#data = open('test.txt').read().split('\n')

SEARCH_MIN = 200000000000000
SEARCH_MAX = 400000000000000
data = open('input.txt').read().split('\n')


def line_intersection(line1, line2):
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])

    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]

    div = det(xdiff, ydiff)
    if div == 0:
        return False

    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return x, y

vectors = []
for line in data:
    vectors.append(tuple([int(thing) for half in line.split(" @ ") for thing in half.split(", ")]))


In [5]:
intersections = set()
checked = set()

for thisv in vectors:
    for thatv in vectors:
        
        if (thatv,thisv) in checked or (thisv,thatv) in checked:
            continue
        if thisv == thatv:
            continue


        a = (thisv[0], thisv[1])
        b = (thisv[0] + 2 * thisv[3], thisv[1] + 2 * thisv[4])
        c = (thatv[0], thatv[1])
        d = (thatv[0] + 2 * thatv[3], thatv[1] + 2 * thatv[4])

        intersection = line_intersection((a, b), (c, d))
        if intersection:
            # if the points intersect

            # check if the intersection is within the bounds that we're got defined
            if SEARCH_MIN < intersection[0] < SEARCH_MAX and SEARCH_MIN < intersection[1] < SEARCH_MAX:
                # check if the interesction is in the direction that the hailstones are moving
                #   We do this because we shouldn't count them if the intersections are where the stones came from
                #   I didn't have this initially, so my number was WAY too high
                if all((intersection[0] - hs[0]) * hs[3] >= 0 and (intersection[1] - hs[1]) * hs[4] >= 0 for hs in (thisv, thatv)):
                    intersections.add(intersection)
        
        checked.add((thisv,thatv))

print(len(intersections))



16939


### HN's solution

Pretty much the same as mine, but he had the magic sauce of checking if the interestion of the lines is in the direction of travel

In [6]:
class Hailstone:
    def __init__(self, sx, sy, sz, vx, vy, vz):
        self.sx = sx
        self.sy = sy
        self.sz = sz
        self.vx = vx
        self.vy = vy
        self.vz = vz
        
        self.a = vy
        self.b = -vx
        self.c = vy * sx - vx * sy
    
    def __repr__(self):
        return "Hailstone{" + f"a={self.a}, b={self.b}, c={self.c}" + "}"

hailstones = [Hailstone(*map(int, line.replace("@", ",").split(","))) for line in open('input.txt')]

total = 0

for i, hs1 in enumerate(hailstones):
    for hs2 in hailstones[:i]:
        a1, b1, c1 = hs1.a, hs1.b, hs1.c
        a2, b2, c2 = hs2.a, hs2.b, hs2.c
        if a1 * b2 == b1 * a2:
            continue
        x = (c1 * b2 - c2 * b1) / (a1 * b2 - a2 * b1)
        y = (c2 * a1 - c1 * a2) / (a1 * b2 - a2 * b1)
        if 200000000000000 <= x <= 400000000000000 and 200000000000000 <= y <= 400000000000000:
            if all((x - hs.sx) * hs.vx >= 0 and (y - hs.sy) * hs.vy >= 0 for hs in (hs1, hs2)):
                total += 1

print(total)

16939
